# Processing:

1) Z-axis signal
2) Rectification
3) Frequency filtering using a fourth-order Butterworth bandpass filter with a cutoff frequency of 18Hz
4) Removal of the first 100 noisy samples
5) Rectification
6) Normalization using z-score
7) Threshold definition (80% of the standard deviation of the signal energy)
8) Peak detection
9) Calculation of heart rate (using the distance between peaks)

-

1) Segnale asse Z
2) Valore assoluto
3) Filtraggio in frequenza tramite Butterworth passa banda quarto ordine f_cutoff=18Hz
4) Rimozione primi 100 campioni sporchi 
5) Valore assoluto
6) Normalizzazione con z-score
7) Definizione threshold (80% della deviazione standard dell'energia del segnale)
8) Indiviuazione picchi
9) Calcolo heart rate (tramite la distanza fra picchi)

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import heartpy as hp

from scipy.signal import savgol_filter
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks
from scipy.stats import zscore

In [ ]:
# Leggi il file excel e suddividi i campioni dei tre assi in 3 vettori
file_path = r'' # file path 1m-50Hz-seduto-67bpm.xlsx 
df = pd.read_excel(file_path)
df.columns = ['acc_x', 'acc_y', 'acc_z']
acc_x = np.array(df['acc_x'])
acc_y = np.array(df['acc_y'])
acc_z = np.array(df['acc_z'])

In [ ]:
# Stampa i primi valori delle tre colonne
print("acc_x:", df['acc_x'].tolist())
print("acc_y:", df['acc_y'].tolist())
print("acc_z:", df['acc_z'].tolist())

In [ ]:
# Segnale asse Z
accelerometer_data_raw = acc_z[2000:3500]
plt.plot(accelerometer_data_raw )

In [ ]:
# Valore assoluto
accelerometer_data_raw = np.abs(accelerometer_data_raw)
plt.plot(accelerometer_data_raw)

In [ ]:
# Filtraggio in frequenza tramite Butterworth passa banda quarto ordine f_cutoff=18Hz
filter_order=4
normalized_cutoff_frequency=18/50
b, a = butter(filter_order, normalized_cutoff_frequency, btype='high', analog=False, output='ba')
signal_high = lfilter(b, a, accelerometer_data_raw)

In [ ]:
# Rimozione primi 100 campioni sporchi 
signal_high=signal_high[100:]
plt.plot(signal_high)

In [ ]:
# Normalizzazione con z-score
signal_high = zscore(signal_high)

In [ ]:
# Definizione threshold (80% della deviazione standard dell'energia del segnale)
signal_high = abs(signal_high)
threshold =(np.std(np.square(signal_high)))*0.8
print(threshold)

In [ ]:
# Individuazione picchi
peaks_filtered, _ = find_peaks(signal_high, height=threshold, distance=25) 
print(peaks_filtered)

In [ ]:
# Plot segnale con picchi evidenziati
plt.plot(signal_high, label='Final Filtered Signal')
plt.plot(peaks_filtered, signal_high[peaks_filtered], 'ro', label='detected peaks')

plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Final Filtered signal with identified peaks')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Determina HR partendo dalla distanza fra picchi ...
interval_filtered = np.diff(peaks_filtered) / 50
print(interval_filtered)
HR_interval = 60 / np.mean(interval_filtered)
print(HR_interval)